In [1]:
import os
import sys
from pathlib import Path

# Force correct project root
project_root = Path("C:/Users/tilak/Desktop/TEXT-SUMMARIZER").resolve()
os.chdir(project_root)
print("Current working directory set to:", os.getcwd())

# Add the `src` folder to Python path
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("SRC path added to sys.path:", src_path)


Current working directory set to: C:\Users\tilak\Desktop\Text-Summarizer
SRC path added to sys.path: C:\Users\tilak\Desktop\Text-Summarizer\src


In [2]:
%pwd

'C:\\Users\\tilak\\Desktop\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        return data_transformation_config



In [6]:
import os
from textsummarizer.logging import logger
from transformers import AutoTokenizer, PegasusTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\tilak\.conda\envs\textSM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoTokenizer, PegasusTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
        if "pegasus" in config.tokenizer_name.lower():
            self.tokenizer = PegasusTokenizer.from_pretrained(config.tokenizer_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name, use_fast=False)

    def convert_examples_to_features(self, example_batch):
      
        input_encodings = self.tokenizer(example_batch['dialogue'],max_length=1024,padding="max_length",truncation=True)

      
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'],max_length=128,padding="max_length",truncation=True)

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }

    def convert(self):
        dataset_sam = load_from_disk(self.config.data_path)
        dataset_sam_pt = dataset_sam.map(self.convert_examples_to_features, batched=True)
        dataset_sam_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [10]:
try:
    config =ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2025-08-24 13:05:13,658]:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-08-24 13:05:13,660]:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-24 13:05:13,661]:INFO:common: created directory at: artifacts]
[2025-08-24 13:05:13,662]:INFO:common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\tilak\.conda\envs\textSM\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 80566.88 examples/s]
